# Decision Trees
This notebook is designed to make an initial exploration of decision trees.  It will use the week 3 loan dataset.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
loans = pd.read_csv("./lending-club-data.csv", dtype={'next_pymnt_d':str, 'desc':str})

Format the target column to reflect safe as 1 and a risky loan as -1.

In [3]:
loans['target'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans', axis=1)

Split the data in train_raw and test_raw set

In [4]:
train_raw, test_raw = train_test_split(loans, test_size=0.2, random_state=1)

Display the percentage of risky and safe loans.

In [5]:
percent_safe_raw = train_raw[train_raw['target'] == 1].size / (1.0 * train_raw.size) * 100
percent_risky_raw = train_raw[train_raw['target'] == -1].size / (1.0 * train_raw.size) * 100

print('Safe: {0:.2f}%'.format(percent_safe_raw))
print('Risky: {0:.2f}%'.format(percent_risky_raw))
print('Size of training set: {0}'.format(len(train_raw.index)))

Safe: 81.16%
Risky: 18.84%
Size of training set: 98085


In [6]:
train_raw

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,target
55546,4177013,5319360,7000,7000,7000,36 months,9.71,224.92,B,B1,...,1.0,1.0,1.0,1,5.09253,20160701T000000,0,1,0,1
99617,1186081,1427424,5300,5300,5300,36 months,14.65,182.82,C,C2,...,1.0,1.0,1.0,0,6.64800,20150301T000000,1,1,1,1
97495,1248384,1484045,20000,20000,20000,36 months,12.12,665.44,B,B3,...,1.0,1.0,1.0,0,10.37050,20150501T000000,1,1,1,1
113058,14569497,16631810,7200,7200,7200,36 months,7.62,224.37,A,A3,...,1.0,1.0,1.0,0,6.73110,20170501T000000,0,1,1,1
64100,4549473,5801857,9600,9600,9600,36 months,18.49,349.43,D,D2,...,1.0,1.0,1.0,0,11.64770,20160501T000000,1,1,1,-1
34876,278075,277896,12000,12000,6278,36 months,10.39,389.41,B,B4,...,1.0,1.0,1.0,0,15.57640,20110301T000000,0,0,1,1
12026,771835,955426,16000,16000,15975,36 months,5.99,486.68,A,A2,...,1.0,1.0,1.0,0,7.30020,20140601T000000,1,1,1,1
95062,1316475,1562041,9075,9075,9075,36 months,15.31,315.97,C,C3,...,1.0,1.0,1.0,0,8.81777,20150601T000000,0,1,1,1
94876,1322778,1567730,10400,10400,10400,36 months,14.65,358.74,C,C2,...,1.0,1.0,1.0,0,13.66630,20150601T000000,1,1,1,-1
94050,1343885,1588483,10000,10000,10000,36 months,13.99,341.73,C,C1,...,1.0,1.0,1.0,0,10.00190,20150601T000000,1,1,1,1


# Effect of Unbalanced data
If there is a class inbalance, there is a chance that it will bias what the majority class will be.  This section explores this concept and shows class imbalance would affect feature selection.

Compute classification error for a feature.

In [14]:
def classification_error(data, feat='', target='output', Verbose=False):
    '''Calculates classification error
    
    Loops through each catagory in the feature and calculates the 
    classification error.  If the feature name is empty, the method
    calculates the classification error of the local root.
    
    Args:
        feat (str): Catagorical variable name or the empty string
        target (str): associated data should be binary
        
    Returns
        float: The classification error.
    '''
    if feat is '': # Root
        num_pos = len(data[data[target] == 1].index)
        num_neg = len(data[data[target] == -1].index)
        if num_pos > num_neg:
            return num_neg / float(len(data.index))
        else:
            return num_pos / float(len(data.index))
    else: # Not the root
        num_error = 0

        bins = data[feat].unique()
        for value in bins.tolist():
            data_in_bin = data[data[feat] == value]
            num_pos = len(data_in_bin[data_in_bin[target] == 1].index)
            num_neg = len(data_in_bin[data_in_bin[target] == -1].index)
            if Verbose:
                print('In feature', value)
                print('Safe: {0:.2f}% with {1} total'.format(num_pos/ float(len(data_in_bin.index)) * 100, num_pos))
                print('Risky: {0:.2f}% with {1} total'.format(num_neg / float(len(data_in_bin.index)) * 100, num_neg))
                print('-------------------')
            if num_pos > num_neg:
                num_error += num_neg
            else:
                num_error += num_pos

        return num_error / float(len(data.index))

For each feature, print the percentage of risky vs safe loans and the classification error for the feature.  Since there is a class imbalance, the classifiction error is the same as the root without any features added even though this is an unlikely senario.

In [16]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
for feature in features:
    print('--------{0}----------'.format(feature))
    print(classification_error(train_raw, feature, 'target', Verbose=True))

--------grade----------
('In feature', 'B')
Safe: 85.38% with 25383 total
Risky: 14.62% with 4346 total
-------------------
('In feature', 'C')
Safe: 79.45% with 19069 total
Risky: 20.55% with 4933 total
-------------------
('In feature', 'A')
Safe: 92.98% with 16631 total
Risky: 7.02% with 1255 total
-------------------
('In feature', 'D')
Safe: 73.20% with 11193 total
Risky: 26.80% with 4098 total
-------------------
('In feature', 'E')
Safe: 68.20% with 4916 total
Risky: 31.80% with 2292 total
-------------------
('In feature', 'F')
Safe: 60.98% with 1896 total
Risky: 39.02% with 1213 total
-------------------
('In feature', 'G')
Safe: 60.00% with 516 total
Risky: 40.00% with 344 total
-------------------
0.188418208697
--------term----------
('In feature', ' 36 months')
Safe: 84.01% with 65733 total
Risky: 15.99% with 12510 total
-------------------
('In feature', ' 60 months')
Safe: 69.91% with 13871 total
Risky: 30.09% with 5971 total
-------------------
0.188418208697
--------ho

We will use a simple method to balance the class.  We will remove rows from the majority class till the classes are balanced.

In [17]:
percentage_to_remove = 1 - (percent_risky_raw / percent_safe_raw)
train_raw_safe = train_raw[train_raw['target'] == 1]

print("Removing {0:.2f}% of the safe set".format(percentage_to_remove*100))

# Ignore the returned test split
train_safe,_ = train_test_split(train_raw_safe, test_size=percentage_to_remove, random_state=1)

# Combine reduced safe set with raw risky set
train = pd.concat([train_safe, train_raw[train_raw['target'] == -1]])

percent_safe = train[train['target'] == 1].size / (1.0 * train.size) * 100
percent_risky = train[train['target'] == -1].size / (1.0 * train.size) * 100

print("Removed {0} items".format(len(train_raw.index)-len(train.index)))
print('Safe: {0:.2f}%'.format(percent_safe))
print('Risky: {0:.2f}%'.format(percent_risky))
print('Size of new training set: {0}'.format(len(train.index)))

Removing 76.78% of the safe set
Removed 61123 items
Safe: 50.00%
Risky: 50.00%
Size of new training set: 36962


Now if we run for classification error again we get better results

In [20]:
for feature in features:
    print('--------{0}----------'.format(feature))
    print(classification_error(train, feature, 'target'))

--------grade----------
0.386829717007
--------term----------
0.423597207943
--------home_ownership----------
0.47251230994
--------emp_length----------
0.485552729831


# Selecting a feature

In [ ]:
def get_class(data, target, value):
    return data[data[target] == value]

In [ ]:
def next_feature(data, features, target):
    '''Performs a single step of feature seletion.
    
    Args:
        data (DataFrame)
        features (array_like): list of features referencing columns in the data
        target (str): name of target column in the data
    
    Note:
        Data target must be a binary catagory where positive is 1 and
        negative is -1.
        
    Todo:
        Add support for non binary classification.
        
    Returns:
        str: The name of feature with the lowest classification error
            or None if the root has the lowest classification error.
    '''
    

In [ ]:
#get_feature_error(train, 'target', 'target', Verbose=True)